### Data Generator

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [ ]:
train_idg = image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True
)

val_idg = image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator = train_idg.flow_from_directory(
    '../Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
val_generator = val_idg.flow_from_directory(
    '../Dataset/test',
    target_size=(224,224),
    batch_size=24,
    class_mode='binary'
)

In [ ]:
from keras.applications import VGG16,ResNet50
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam

### Vgg16 Model

In [ ]:
model = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [ ]:
av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(1,activation='sigmoid')(d2)

In [ ]:
vgg = Model(model.input,out)
print(len(vgg.layers))

In [ ]:
for i in range(17): 
    vgg.layers[i].trainable = False
vgg.summary()

for i in range(169):
    resnet50.layers[i].trainable = False
    
resnet50.summary()

In [ ]:
adam = Adam(lr=0.001)
vgg.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

- Train Model

In [ ]:
from keras.callbacks import ModelCheckpoint
import os
import glob

In [ ]:
if not os.path.exists('../Weights/Vgg'):
    os.mkdir('../Weights/Vgg')
    print('Directory created')
else:
    print('Already Created')

In [ ]:
files = glob.glob('../Weights/Vgg/*')
for f in files:
    os.remove(f)

In [ ]:
checkpoint = ModelCheckpoint(
    '../Weights/Vgg/model_{epoch}_{val_accuracy:.2f}.h5',
    monitor='val_accuracy', 
    verbose=1, 
    mode='auto'
)

In [ ]:
hist = vgg.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=3,
    callbacks=checkpoint
)

In [ ]:
x,y=val_generator.next()

In [ ]:
vgg.evaluate(x,y)

- Save Model History

In [ ]:
import numpy as np
import os

In [ ]:
if not os.path.exists('../Model_history'):
    os.mkdir('../Model_history')
    print('Directory created')
else:
    print('Already Created')

In [ ]:
np.save('../Model_history/vgg_file.npy',hist.history)
read_dictionary = np.load('../Model_history/vgg_file.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())

### Resnet50 Model

In [7]:
model = ResNet50(include_top=False,weights='imagenet',input_shape = (224,224,3))

In [9]:
av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(1,activation='sigmoid')(d2)

In [11]:
resnet = Model(model.input,out)

for i in range(169):
    resnet.layers[i].trainable = False
    
adam = Adam(lr=0.0005)
resnet.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [13]:
if not os.path.exists('../Weights/Resnet50'):
    os.mkdir('../Weights/Resnet50')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [16]:
files = glob.glob('../Weights/Resnet50/*')
for f in files:
    os.remove(f)

In [17]:
checkpoint = ModelCheckpoint(
    '../Weights/Resnet50/model_{epoch}_{val_accuracy:.2f}.h5',
    monitor='val_accuracy', 
    verbose=1, 
    mode='auto'
)

In [ ]:
hist = resnet.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=3,
    #callbacks=checkpoint
)

Epoch 1/10
9/9 [==============================] - 60s 7s/step - loss: 0.9533 - accuracy: 0.5311 - val_loss: 1.5049 - val_accuracy: 0.5882
Epoch 2/10
9/9 [==============================] - 59s 7s/step - loss: 0.5542 - accuracy: 0.7326 - val_loss: 1.6038 - val_accuracy: 0.5735
Epoch 3/10
9/9 [==============================] - 61s 7s/step - loss: 0.4121 - accuracy: 0.8388 - val_loss: 1.7360 - val_accuracy: 0.5735
Epoch 4/10
9/9 [==============================] - 63s 7s/step - loss: 0.2835 - accuracy: 0.8974 - val_loss: 1.9664 - val_accuracy: 0.5882
Epoch 5/10
9/9 [==============================] - 64s 7s/step - loss: 0.2313 - accuracy: 0.8828 - val_loss: 2.5582 - val_accuracy: 0.6176
Epoch 6/10
9/9 [==============================] - 68s 8s/step - loss: 0.3274 - accuracy: 0.8828 - val_loss: 1.9708 - val_accuracy: 0.6176
Epoch 7/10
9/9 [==============================] - 64s 7s/step - loss: 0.2838 - accuracy: 0.9048 - val_loss: 1.8681 - val_accuracy: 0.5882
Epoch 8/10
5/9 [===============>..